In [1]:
%matplotlib inline
%load_ext nb_black

<IPython.core.display.Javascript object>


# First level analysis of a complete BIDS dataset from openneuro

---

**Taken and adapted from the nilearn documentation**

See the original [here]( ??? )

---

Full step-by-step example of fitting a GLM to perform a first level analysis
in an openneuro BIDS dataset. We demonstrate how BIDS derivatives can be
exploited to perform a simple one subject analysis with minimal code.
Details about the BIDS standard are available at http://bids.neuroimaging.io/.
We also demonstrate how to download individual groups of files from the
Openneuro s3 bucket.

More specifically:

1. Download an fMRI BIDS dataset with derivatives from openneuro.
2. Extract first level model objects automatically from the BIDS dataset.
3. Demonstrate Quality assurance of Nistats estimation against available FSL.
   estimation in the openneuro dataset.
4. Display contrast plot and uncorrected first level statistics table report.



To run this example, you must launch IPython via ``ipython
--matplotlib`` in a terminal, or use the Jupyter notebook.
    :depth: 1


## Fetch openneuro BIDS dataset

We download one subject from the stopsignal task in the [ds000030](https://openneuro.org/datasets/ds000030/versions/1.0.0) V4 BIDS
dataset available in openneuro.


The preprocessed data described in are available at https://legacy.openfmri.org/dataset/ds000030/ and via Amazon Web Services S3 protocol at: s3://openneuro/ds000030/ds000030_R1.0.5/uncompressed/derivatives/

TODO:

    - download derivatives with datalad

This dataset contains the necessary information to run a statistical analysis
using Nilearn. The dataset also contains statistical results from a previous
FSL analysis that we can employ for comparison with the Nilearn estimation.



In [3]:
!datalad clone https://github.com/OpenNeuroDatasets/ds000030.git ../inputs/ds000030/

Clone attempt:   0%|              | 0.00/2.00 [00:00<?, ? Candidate locations/s]
Enumerating: 0.00 Objects [00:00, ? Objects/s]
                                              
Receiving:   0%|                             | 0.00/66.4k [00:00<?, ? Objects/s]
Receiving:   3%|▌                   | 1.99k/66.4k [00:00<00:03, 17.2k Objects/s]
Receiving:  12%|██▍                 | 7.97k/66.4k [00:00<00:02, 21.6k Objects/s]
Receiving:  20%|████                | 13.3k/66.4k [00:00<00:02, 26.1k Objects/s]
Receiving:  36%|███████▏            | 23.9k/66.4k [00:00<00:01, 33.4k Objects/s]
Receiving:  51%|██████████▏         | 33.9k/66.4k [00:00<00:00, 41.3k Objects/s]
Receiving:  70%|██████████████      | 46.5k/66.4k [00:00<00:00, 51.4k Objects/s]
Receiving:  84%|████████████████▊   | 55.8k/66.4k [00:00<00:00, 59.0k Objects/s]
Receiving:  97%|███████████████████▍| 64.4k/66.4k [00:01<00:00, 37.0k Objects/s]
                                                                                
Resolving:   0%

<IPython.core.display.Javascript object>

In [11]:
!datalad addurls s3://openneuro/ds000030/ds000030_R1.0.5/uncompressed/derivatives/

usage: datalad addurls [-h] [-d DATASET] [-t TYPE] [-x REGEXP] [-m FORMAT]
                       [--message MESSAGE] [-n] [--fast]
                       [--ifexists {overwrite|skip}] [--missing-value VALUE]
                       [--nosave] [--version-urls] [-c PROC]
                       URL-FILE URL-FORMAT FILENAME-FORMAT
datalad addurls: error: the following arguments are required: URL-FORMAT, FILENAME-FORMAT


<IPython.core.display.Javascript object>

In [7]:
!tree -d ../inputs/ds000030/

../inputs/ds000030/
├── phenotype
├── sub-10159
│   ├── anat
│   ├── beh
│   ├── dwi
│   └── func
├── sub-10171
│   ├── anat
│   ├── beh
│   ├── dwi
│   └── func
├── sub-10189
│   ├── anat
│   ├── beh
│   ├── dwi
│   └── func
├── sub-10193
│   ├── anat
│   ├── dwi
│   └── func
├── sub-10206
│   ├── anat
│   ├── beh
│   ├── dwi
│   └── func
├── sub-10217
│   ├── anat
│   ├── beh
│   ├── dwi
│   └── func
├── sub-10225
│   ├── anat
│   ├── beh
│   ├── dwi
│   └── func
├── sub-10227
│   ├── anat
│   ├── beh
│   ├── dwi
│   └── func
├── sub-10228
│   ├── anat
│   ├── beh
│   ├── dwi
│   └── func
├── sub-10235
│   ├── anat
│   ├── beh
│   ├── dwi
│   └── func
├── sub-10249
│   ├── anat
│   ├── beh
│   ├── dwi
│   └── func
├── sub-10269
│   ├── anat
│   ├── beh
│   ├── dwi
│   └── func
├── sub-10271
│   ├── anat
│   ├── beh
│   ├── dwi
│   └── func
├── sub-10273
│   ├── anat
│   ├── beh
│   ├── dwi
│   └── func
├── sub-10274
│   ├── anat
│   ├── beh
│   ├── dwi
│   └── func
├── sub-10280
│   

<IPython.core.display.Javascript object>

In [10]:
!tree ../inputs/ds000030/sub-10159

../inputs/ds000030/sub-10159
├── anat
│   ├── sub-10159_T1w.json
│   └── sub-10159_T1w.nii.gz -> ../../.git/annex/objects/KM/vk/MD5E-s11637742--75cab8005361c2504ba5a7f02ecbacd7.nii.gz/MD5E-s11637742--75cab8005361c2504ba5a7f02ecbacd7.nii.gz
├── beh
│   ├── sub-10159_task-stopsignaltraining_beh.json
│   └── sub-10159_task-stopsignaltraining_events.tsv
├── dwi
│   ├── sub-10159_dwi.bval
│   ├── sub-10159_dwi.bvec
│   ├── sub-10159_dwi.json
│   └── sub-10159_dwi.nii.gz -> ../../.git/annex/objects/Qm/mj/MD5E-s32000307--7b1e3890b74b9a9d96729ea967c69d95.nii.gz/MD5E-s32000307--7b1e3890b74b9a9d96729ea967c69d95.nii.gz
└── func
    ├── sub-10159_task-bart_bold.json
    ├── sub-10159_task-bart_bold.nii.gz -> ../../.git/annex/objects/W1/V4/MD5E-s42078486--6ae43b435234a9f9179f95a216be9ba9.nii.gz/MD5E-s42078486--6ae43b435234a9f9179f95a216be9ba9.nii.gz
    ├── sub-10159_task-bart_events.tsv
    ├── sub-10159_task-rest_bold.json
    ├── sub-10159_task-rest_bold.nii.gz -> ../../.git/annex/objects/Jj/MK/

<IPython.core.display.Javascript object>

In [8]:
from bids import BIDSLayout

data_folder = "../inputs/ds000030"
layout = BIDSLayout(data_folder)

KeyboardInterrupt: 

<IPython.core.display.Javascript object>

## Obtain FirstLevelModel objects automatically and fit arguments
From the dataset directory we automatically obtain FirstLevelModel objects
with their subject_id filled from the BIDS dataset. Moreover we obtain,
for each model, the list of run images and their respective events and
confound regressors. Those are inferred from the confounds.tsv files
available in the BIDS dataset.
To get the first level models we have to specify the dataset directory,
the task_label and the space_label as specified in the file names.
We also have to provide the folder with the desired derivatives, that in this
case were produced by the fmriprep BIDS app.



In [ ]:
from nilearn.glm.first_level import first_level_from_bids
task_label = 'stopsignal'
space_label = 'MNI152NLin2009cAsym'
derivatives_folder = 'derivatives/fmriprep'
models, models_run_imgs, models_events, models_confounds = \
    first_level_from_bids(data_dir, task_label, space_label,
                          smoothing_fwhm=5.0,
                          derivatives_folder=derivatives_folder)

Access the model and model arguments of the subject and process events.



In [ ]:
model, imgs, events, confounds = (
    models[0], models_run_imgs[0], models_events[0], models_confounds[0])
subject = 'sub-' + model.subject_label

import os
from nilearn._utils.glm import get_design_from_fslmat
fsl_design_matrix_path = os.path.join(
    data_dir, 'derivatives', 'task', subject, 'stopsignal.feat', 'design.mat')
design_matrix = get_design_from_fslmat(
    fsl_design_matrix_path, column_names=None)

We identify the columns of the Go and StopSuccess conditions of the
design matrix inferred from the FSL file, to use them later for contrast
definition.



In [ ]:
design_columns = ['cond_%02d' % i for i in range(len(design_matrix.columns))]
design_columns[0] = 'Go'
design_columns[4] = 'StopSuccess'
design_matrix.columns = design_columns

## First level model estimation (one subject)
We fit the first level model for one subject.



In [ ]:
model.fit(imgs, design_matrices=[design_matrix])

Then we compute the StopSuccess - Go contrast. We can use the column names
of the design matrix.



In [ ]:
z_map = model.compute_contrast('StopSuccess - Go')

We show the agreement between the Nilearn estimation and the FSL estimation
available in the dataset.



In [ ]:
import nibabel as nib
fsl_z_map = nib.load(
    os.path.join(data_dir, 'derivatives', 'task', subject, 'stopsignal.feat',
                 'stats', 'zstat12.nii.gz'))

from nilearn import plotting
import matplotlib.pyplot as plt
from scipy.stats import norm
plotting.plot_glass_brain(z_map, colorbar=True, threshold=norm.isf(0.001),
                          title='Nilearn Z map of "StopSuccess - Go" (unc p<0.001)',
                          plot_abs=False, display_mode='ortho')
plotting.plot_glass_brain(fsl_z_map, colorbar=True, threshold=norm.isf(0.001),
                          title='FSL Z map of "StopSuccess - Go" (unc p<0.001)',
                          plot_abs=False, display_mode='ortho')
plt.show()

from nilearn.plotting import plot_img_comparison
plot_img_comparison([z_map], [fsl_z_map], model.masker_,
                    ref_label='Nilearn', src_label='FSL')
plt.show()

## Simple statistical report of thresholded contrast
We display the contrast plot and table with cluster information



In [ ]:
from nilearn.plotting import plot_contrast_matrix
plot_contrast_matrix('StopSuccess - Go', design_matrix)
plotting.plot_glass_brain(z_map, colorbar=True, threshold=norm.isf(0.001),
                          plot_abs=False, display_mode='z',
                          figure=plt.figure(figsize=(4, 4)))
plt.show()

We can get a latex table from a Pandas Dataframe for display and publication
purposes



In [ ]:
from nilearn.reporting import get_clusters_table
print(get_clusters_table(z_map, norm.isf(0.001), 10).to_latex())

## Generating a report
Using the computed FirstLevelModel and contrast information,
we can quickly create a summary report.



In [ ]:
from nilearn.reporting import make_glm_report

report = make_glm_report(model=model,
                         contrasts='StopSuccess - Go',
                         )

We have several ways to access the report:



In [ ]:
# report  # This report can be viewed in a notebook
# report.save_as_html('report.html')
# report.open_in_browser()